In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from keras.models import Model
# Enable mixed precision
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

# Define the path to the dataset and batch size
path = r"C:\Users\Rajarshi\Downloads\Documents\aerospace_images"
batch_size = 32  # Increased batch size

# Step 1: Set up data generators
image_generator = ImageDataGenerator(rescale=1/255, 
                                     horizontal_flip=True,
                                     zoom_range=0.2, 
                                     validation_split=0.2)

image_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                    directory=path,
                                                    shuffle=True,
                                                    target_size=(224, 224),
                                                    subset="training",
                                                    class_mode="categorical")

validation_dataset = image_generator.flow_from_directory(batch_size=batch_size,
                                                         directory=path,
                                                         shuffle=True,
                                                         target_size=(224, 224),
                                                         subset="validation",
                                                         class_mode="categorical")

IMG_SHAPE = (224, 224, 3)
num_of_classes = 7  # Assuming this is the correct number of categories

Found 6225 images belonging to 7 classes.
Found 1553 images belonging to 7 classes.


In [9]:
# Step 2: Load a pre-trained  model without the top layer

# Include pre-processing for image size expected by ResNet50 (typically 224x224)
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # 3 for RGB channels

for layer in resnet_model.layers:
  layer.trainable = False

In [10]:
# Step 3: Build the model with a custom top that includes dropout
x = resnet_model.output
x = Flatten()(x)  # Flatten the output of the pre-trained model
x = Dense(units=1024, activation='relu')(x)  # Add a dense layer with ReLU activation
predictions = Dense(units=num_of_classes , activation='softmax')(x)  # Final output layer with softmax for multi-class classification
# Create a new model with the pre-trained base and the added layers
model = Model(inputs=resnet_model.input, outputs=predictions)
model._name = "Air_res50"



In [11]:
# Step 4: Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Print the model summary
print("Initial model summary (with frozen layers):")
print(model.summary())

Initial model summary (with frozen layers):


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 126,356,359 (482.01 MB)

 Trainable params: 102,768,647 (392.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

None


In [12]:
# Step 6: Train the model with frozen base layers
hist_initial = model.fit(
    image_dataset,
    epochs=6,  # Reduced epochs for initial training
    validation_data=validation_dataset,
    verbose=1
)

Epoch 1/6


C:\Users\Rajarshi\anaconda3\envs\py312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


195/195 ━━━━━━━━━━━━━━━━━━━━ 506s 3s/step - accuracy: 0.2388 - loss: 13.6446 - val_accuracy: 0.3110 - val_loss: 1.8519
Epoch 2/6
183/195 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.3266 - loss: 1.8583

KeyboardInterrupt: 

In [ ]:
# # Step 7: Unfreeze some top layers of the base model for fine-tuning
# for layer in base_model.layers[-4:]:  # Adjust the number of layers to unfreeze as needed
#     layer.trainable = True

# # Recompile the model after unfreezing
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Print the model summary after unfreezing layers
# print("Model summary after unfreezing some layers:")
# print(model.summary())

# # Step 8: Continue training the model with fine-tuning
# hist_fine_tune = model.fit(
#     image_dataset,
#     epochs=10,  # Additional epochs for fine-tuning, reduced for speed
#     validation_data=validation_dataset,
#     verbose=1
# )


In [ ]:
model.save("C:\Users\Rajarshi\Downloads\Programs\air_res50.h5")